# Reading 13Fs

### Importing libraries

In [46]:
import codecs
import urllib2
import bs4 as bs
from bs4 import SoupStrainer
import pandas as pd

In [19]:
import urllib2
import time
from urllib import FancyURLopener  # This is library that helps us create the headless browser
from random import choice #This library helps pick a random item from a list

user_agents = [
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
]

### Searching for CIKs

In [3]:
wildcards = []
CIK = []
Fund_Name = []
Vowels = ['a','e','i','o','u']

for i in Vowels:
    wildcards.append(i + "*")

In [4]:
for item in wildcards:  
    url = 'https://www.sec.gov/cgi-bin/series?company='+item+'&CIK=&start=1&count=500'
    html = urllib2.urlopen(url).read()
    
    last_page = False
    
    counter = 0  #only use this if you want to stop at certain page.
    
    while last_page == False:
    
        index = html.find("<table summary=") #going closer to the results table
        html2 = html[index:]

        #SAME CODE AS BEFORE STARTS HERE
        while html2.find("a class=\"search\" href=\"/cgi-bin/browse-edgar?CIK=") != -1: 

            #We are running a while loop because we want to get everything in the page
            index = html2.find("a class=\"search\" href=\"/cgi-bin/browse-edgar?CIK=")
            remaining = html2[index:]

            start = remaining.find(">")
            end = remaining.find("<")
            CIK.append(remaining[start+1:end])

            remaining = remaining[end:]
            index = remaining.find("</td>")
            remaining = remaining[index:]

            index = remaining.find("action=getcompany&amp;scd=series")
            remaining = remaining[index:]

            start = remaining.find(">")
            end = remaining.find("<")
            Fund_Name.append(remaining[start+1:end].replace(" ", "_"))

            remaining = remaining[end:]
            index = remaining.find("</td>")
            html2 = remaining[index:]
        #SAME CODE AS BEFORE ENDS
        
        index = html.find('count=500">[NEXT]')
        
        if index == -1:
            last_page == True
        
        else:
        
            html2 = html[index-20:index+100]
            start=html2.find('start=')+6
            end = html2.find('&amp;count=')
            start_number = html2[start:end]

            url = 'https://www.sec.gov/cgi-bin/series?company='+item+'&CIK=&start='+start_number+'&count=500' # count=500 is to page this

            html = urllib2.urlopen(url).read()
        
        counter += 1
        if counter == 10:
            last_page = True    #only use this to limit the number of pages to go through.  You can drop this later

In [5]:
Output_File = open("Mutual_Fund_CIK.txt", "w")

Output_File.write("CIK\tCompany\n")

for x in range(0, len(CIK)):
    Output_File.write(CIK[x] + '\t' + Fund_Name[x] + '\t\n')

Output_File.close()

In [6]:
Input_File = open("Mutual_Fund_CIK.txt", "r")

Output_File2 = open("Mutual_Fund_CIK_Sorted.txt", "w")

listLines = []

for line in Input_File:

    if line in listLines:
        continue
# this portion of the code tells us the for loop that if a line is already there in the list, skip the line or else add it to the list
    else:
        Output_File2.write(line)
        listLines.append(line)

Output_File2.close()

Input_File.close()

### Searching for 13Fs

In [7]:
CIK = []
CIK_13F = []
weblinks = []
fund_name = []
name = []


input_file = open('Mutual_Fund_CIK_Sorted.txt', 'r')
rows = input_file.readlines()
input_file.close()

for i in range(1, len(rows)):
    columns = rows[i].split('\t')
    CIK.append(columns[0])
    fund_name.append(columns[1])

In [8]:
for item in CIK:
    
    link = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=' + item + '&type=13F-HR&dateb=&count=100&scd=filings'
    
    class MyOpener(FancyURLopener, object):
        version = choice(user_agents)
    
    myopener = MyOpener()
    page=myopener.open(link)
    
    html = page.read()
            
    index = html.find('a href=\"/Archives/edgar/data/')
    html2 = html[index:]

    while html2.find('a href=\"/Archives/edgar/data/') != -1:

        start = html2.find('a href=\"/Archives/edgar/data/')
        remaining = html2[start:]
        end = remaining.find('id=\"documentsbutton\"')
        weblinks.append(remaining[8:end-2])
        remaining = remaining[end:]

        index = remaining.find('</td>')
        html2 = remaining[index:]

        CIK_13F.append(item)
        name.append(fund_name[CIK.index(item)])

        
        
Output_File = open("Mutual_Fund_13F_links.txt", "w")

Output_File.write("CIK\tCompany\tLinks\n")

for x in range(0, len(name)):
    Output_File.write(CIK_13F[x] + '\t' + name[x] + '\t' + weblinks[x] + '\t' + '\n')

Output_File.close()

### Searching for .txt and .xml files

In [10]:
CIK = []
Name = []
Link = []
Urls = []
date = []
info_table = []
info_type = []

In [11]:
#Reading the links of indivudual 13F HR tables from the previous life
input_file = open('Mutual_Fund_13F_links.txt', 'r')

rows = input_file.readlines()
input_file.close()

for i in range(1, len(rows)):
    columns = rows[i].split('\t')
    CIK.append(columns[0])
    Name.append(columns[1])
    Link.append(columns[2])

for item in Link:
    temp_url = "https://www.sec.gov" + item
    Urls.append(temp_url)

In [12]:
for page in Urls:
    html = urllib2.urlopen(page).read()
    
    date_index = html.find('div class=\"infoHead\">Filing Date<')
    date_html = html[date_index:]
    date_index2 = date_html.find('class=\"info\">')
    date_html = date_html[date_index2:]
    start = date_html.find(">")
    end = date_html.find("<")
    date.append(date_html[start+1:end])  # This gets us the date
    
    if html.find('td scope=\"row\">2<') != -1: # By this condition we check whether there is a xml version. For an xml version there are two items present in each table in the page
        
        index = html.find('td scope=\"row\">2<')
        html2 = html[index:]
    
        start = html2.find('a href=\"/Archives/edgar/data/')
        remaining = html2[start:]
        end = remaining.find('>')
    
        info_table.append(remaining[8:end-1]) # we collect the link
    
        info_type.append('xml') # we mention that this is an xml link
            
    else: # Or else, we get the idea that the info table is in text format
        
        index = html.find('td scope=\"row\">Complete submission text file')
        html2 = html[index:]
        
        start = html2.find('a href=\"/Archives/edgar/data/')
        remaining = html2[start:]
        end = remaining.find('>')
    
        info_table.append(remaining[8:end-1]) 

        info_type.append('text') # we mention that this is a text link

In [13]:
# Next we write the all these info table links down to a CSV file:

Output_File = open("Mutual_Fund_Info_Table_Link.txt", 'w')

Output_File.write("CIK, Name, Date, Information Table Links, File Type, \n")

for x in range(0, len(info_table)):
    Output_File.write(CIK[x] + '\t' + Name[x] + '\t' + date[x] + '\t' + info_table[x] + '\t' + info_type[x] + '\n')

Output_File.close()

### Putting .xml files' data into a dataframe

In [15]:
mutual_fund_name = []
info_date = []
name_of_issuer = []
title_of_class = []
CUSIP = []
Value = []
Shares = []
SHRS_or_PRN = []
SH_PRN = []
PUTorCALL = []
Inv_Discretion = []
Other_Manager = []
Authority_Sole = []
Authority_Shared = []
Authority_None = []

In [17]:
CIK = []
xml_date = []
xml_Name = []
xml_link = []

input_file = open('Mutual_Fund_Info_Table_Link.txt', 'r')

rows = input_file.readlines()
input_file.close()

for i in range(1, len(rows)):
    columns = rows[i].split('\t')
    
    CIK.append(columns[0])
    
    if columns[4][:-1] == "xml":
        xml_link.append('https://www.sec.gov' + columns[3])
        xml_Name.append(columns[1])
        xml_date.append(columns[2])

In [22]:
for link_number in range(0,len(xml_link)-1):
    
    myopener = MyOpener()
    page=myopener.open(xml_link[link_number])
    html = page.read()
    
    soup = bs.BeautifulSoup(html, 'lxml')
    
    infotable = soup.find_all("table", summary = "Form 13F-NT Header Information")
    
    for row in infotable[0].find_all('tr')[3:]:
        columns = row.find_all('td')
        mutual_fund_name.append(xml_Name[link_number])
        info_date.append(xml_date[link_number])
        name_of_issuer.append(columns[0].getText())
        title_of_class.append(columns[1].getText())
        CUSIP.append(columns[2].getText())
        Value.append(columns[3].getText())
        SHRS_or_PRN.append(columns[4].getText())
        SH_PRN.append(columns[5].getText())
        PUTorCALL.append(columns[6].getText())
        Inv_Discretion.append(columns[7].getText())
        Other_Manager.append(columns[8].getText())
        Authority_Sole.append(columns[9].getText())
        Authority_Shared.append(columns[10].getText())
        Authority_None.append(columns[11].getText())

In [53]:
final1 = pd.DataFrame(
    {'Company': name_of_issuer,
     'Filing Type': title_of_class,
     'CUSIP': CUSIP,
     'Market Value': Value,
     'Quantity': SHRS_or_PRN,
     'Mutual Fund': mutual_fund_name,
     'Date': info_date
    })

### Putting .txt files' data into a file

In [32]:
CIK = []
txt_date = []
txt_Name = []
txt_link = []

input_file = open('Mutual_Fund_Info_Table_Link.txt', 'r')

rows = input_file.readlines()
input_file.close()

for i in range(1, len(rows)):
    columns = rows[i].split('\t')
    
    CIK.append(columns[0])
    
    if columns[4][:-1] == "text":
        txt_link.append('https://www.sec.gov' + columns[3])
        txt_Name.append(columns[1])
        txt_date.append(columns[2])

In [40]:
def df2(index1,index2, x, position):
    listus=[]
    for d in range(index1,index2):
        trym = []
        for e in range(0,5): 
            temp_var = x[d][position[e]:position[e+1]].strip()
            if (e != 0) & (e!= 1):
                temp_var = re.sub(r'\W+', '', temp_var)
            trym.append(temp_var)


        if((len(trym[2])== 9) & (len(trym[0]) > 0) & (len(trym[1]) > 0) & (len(trym[3]) > 0) & (len(trym[4]) > 0) ):
            trym.append(txt_Name[link])
            trym.append(txt_date[link])
            listus.append(trym) 
        
    return(listus)

In [44]:
import re
my_list = []

for link in range(0,len(txt_link)):    ###Looping for all the links
    myopener = MyOpener()
    page=myopener.open(str(txt_link[link]))   ####Opening the link
    html = page.read()
    x = html.splitlines(0)    ##### Creating a list of lines
    index = 0
    indexes = []
    for i in range(0,len(x)):
        
        tempo = []
        if x[i].replace(" ", "").find('<C><C><C><C><C><C>') != -1:   ####This is to handle the condition where multiple pages exist as each page has <C>
            index = i
            y = []
            #### Finding all the indexes of the <C>
            for m in re.finditer('<C>', x[index]):
                if m.start() == 0:
                    continue
                else:
                    y.append(m.start())
            y = [0] + y
            tempo.append(index)   
            tempo.append(y)
            indexes.append(tempo)
    if len(indexes) == 1: #if there is only one chunk of data
        index2 = len(x)
        
        my_list = my_list + df2(indexes[0][0],index2,x,indexes[0][1])
    if len(indexes) >1: #if there are multiple chunks
       
        
        for g in range(0,len(indexes)):
            index1 = indexes[g][0] #appending chunk one by one
            if g != len(indexes)-1:
                index2 = indexes[g+1][0]
            if g == len(indexes)-1:
                index2 = len(x)
               
            my_list = my_list + df2(index1,index2,x,indexes[g][1])

In [51]:
final2 = pd.DataFrame(my_list)
final2.columns = ["Company", "Filing Type", "CUSIP",  "Market Value", "Quantity", "Mutual Fund", "Date"]

### Final dataframe and file

In [55]:
final = [final1,final2]

In [56]:
result = pd.concat(final)

In [57]:
result.to_csv("All_Mutual_Funds.csv")

,CUSIP,Company,Date,Filing Type,Market Value,Mutual Fund,Quantity
0,2824100,ABBOTT LABS,2015-11-10,COM,266,Adirondack_Funds,"6,625"
1,00287Y109,ABBVIE INC,2015-11-10,COM,343,Adirondack_Funds,"6,300"
2,13817101,ALCOA INC,2015-11-10,COM,442,Adirondack_Funds,"45,800"
3,01988P108,ALLSCRIPTS HEALTHCARE SOLUTN,2015-11-10,COM,"4,328",Adirondack_Funds,"349,059"
4,00206R102,AT&T INC,2015-11-10,COM,450,Adirondack_Funds,"13,800"
5,05275N205,AUTOBYTEL INC,2015-11-10,COM NEW,"2,938",Adirondack_Funds,"175,170"
6,54540109,AXCELIS TECHNOLOGIES INC,2015-11-10,COM,"3,567",Adirondack_Funds,"1,335,963"
7,00246W103,AXT INC,2015-11-10,COM,532,Adirondack_Funds,"277,040"
8,06846N104,BARRETT BILL CORP,2015-11-10,COM,189,Adirondack_Funds,"57,239"
9,08160H101,BENCHMARK ELECTRS INC,2015-11-10,COM,"1,222",Adirondack_Funds,"56,173"
